# Pickling cuML Models for Persistence

This notebook demonstrates simple pickling of both single-GPU and multi-GPU cuML models for persistence

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Single GPU Model Pickling

All single-GPU estimators are pickleable. The following example demonstrates the creation of a synthetic dataset, training, and pickling of the resulting model for storage. Trained single-GPU models can also be used to distribute the inference on a Dask cluster, which the `Distributed Model Pickling` section below demonstrates.

In [2]:
from cuml.datasets import make_blobs

X, y = make_blobs(n_samples=50,
                  n_features=10,
                  centers=5,
                  cluster_std=0.4,
                  random_state=0)

In [3]:
from cuml.cluster import KMeans

model = KMeans(n_clusters=5)

model.fit(X)

KMeans()

In [4]:
import pickle

pickle.dump(model, open("kmeans_model.pkl", "wb"))

In [5]:
model = pickle.load(open("kmeans_model.pkl", "rb"))

In [6]:
model.cluster_centers_

array([[ 5.2615476 , -4.0487256 ,  4.464928  , -2.9367518 ,  3.5061095 ,
        -4.0168314 , -3.463885  ,  6.078449  , -6.953326  , -1.004144  ],
       [-4.2439985 ,  5.6107073 , -5.669777  , -1.7957245 , -9.255528  ,
         0.71774375,  4.4435906 , -2.8747153 , -5.0900965 ,  9.684121  ],
       [ 4.6749854 ,  8.213465  , -9.07572   ,  9.568374  ,  8.454807  ,
        -1.2327975 ,  3.3903713 , -7.828242  , -0.8454461 ,  0.6288572 ],
       [-3.008261  ,  4.6259604 , -4.483249  ,  2.228457  ,  1.643532  ,
        -2.4505196 , -5.2582016 , -1.6679403 , -7.9857535 ,  2.8311467 ],
       [-5.6072407 ,  2.2695985 , -3.7516532 , -1.8182005 , -5.143028  ,
         7.599363  ,  2.8252366 ,  8.773042  ,  1.6198314 ,  1.1772048 ]],
      dtype=float32)

## Distributed Model Pickling

The distributed estimator wrappers inside of the `cuml.dask` are not intended to be pickled directly. The Dask cuML estimators provide a function `get_combined_model()`, which returns the trained single-GPU model for pickling. The combined model can be used for inference on a single-GPU, and the `ParallelPostFit` wrapper from the [Dask-ML](https://ml.dask.org/meta-estimators.html) library can be used to perform distributed inference on a Dask cluster.

In [7]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: LocalCUDACluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 251.79 GiB
Comm: tcp://127.0.0.1:36137,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 251.79 GiB
Comm: tcp://127.0.0.1:37569,Total threads: 1
Dashboard: http://127.0.0.1:33033/status,Memory: 251.79 GiB
Nanny: tcp://127.0.0.1:38189,


In [8]:
from cuml.dask.datasets import make_blobs

n_workers = len(client.scheduler_info()["workers"].keys())

X, y = make_blobs(n_samples=5000, 
                  n_features=30,
                  centers=5, 
                  cluster_std=0.4, 
                  random_state=0,
                  n_parts=n_workers*5)

X = X.persist()
y = y.persist()

In [9]:
from cuml.dask.cluster import KMeans

dist_model = KMeans(n_clusters=5)

In [10]:
dist_model.fit(X)

In [11]:
import pickle

single_gpu_model = dist_model.get_combined_model()
pickle.dump(single_gpu_model, open("kmeans_model.pkl", "wb"))

In [12]:
single_gpu_model = pickle.load(open("kmeans_model.pkl", "rb"))

In [13]:
single_gpu_model.cluster_centers_

array([[-2.8796384e+00,  4.4348750e+00, -4.4264822e+00,  2.3959134e+00,
         1.7286433e+00, -2.4991984e+00, -5.1794519e+00, -1.6771448e+00,
        -8.1329165e+00,  2.6659231e+00, -4.3131094e+00,  5.5827799e+00,
        -5.7318306e+00, -1.7427168e+00, -9.3456144e+00,  7.1365577e-01,
         4.4255500e+00, -2.9118373e+00, -4.9467444e+00,  9.6786394e+00,
         8.4222708e+00, -6.2710257e+00, -6.3596516e+00,  1.9645505e+00,
         4.1715999e+00, -9.1683645e+00,  4.6156673e+00,  8.7916479e+00,
         6.8754425e+00,  2.2288759e+00],
       [-6.9536943e+00, -9.7635860e+00, -6.5648251e+00, -4.3536144e-01,
         6.0998120e+00,  3.7550366e+00, -3.9558544e+00,  6.1595521e+00,
        -1.8599317e+00,  5.0400319e+00, -6.8397541e+00,  1.3435427e+00,
         8.9749022e+00, -9.9621761e-01,  9.6651907e+00,  9.8009682e+00,
        -8.6188345e+00,  5.9978375e+00,  2.2295940e+00, -3.6477706e+00,
         7.0758505e+00, -7.3772254e+00, -5.3214216e+00, -6.9927936e+00,
        -7.9296317e+00,

## Exporting cuML Random Forest models for inferencing on machines without GPUs

Starting with cuML version 21.06, you can export cuML Random Forest models and run predictions with them on machines without an NVIDIA GPUs. The [Treelite](https://github.com/dmlc/treelite) package defines an efficient exchange format that lets you portably move the cuML Random Forest models to other machines. We will refer to the exchange format as "checkpoints."

Here are the steps to export the model:

1. Call `to_treelite_checkpoint()` to obtain the checkpoint file from the cuML Random Forest model.

In [14]:
from cuml.ensemble import RandomForestClassifier as cumlRandomForestClassifier
from sklearn.datasets import load_iris
import numpy as np

X, y = load_iris(return_X_y=True)
X, y = X.astype(np.float32), y.astype(np.int32)
clf = cumlRandomForestClassifier(max_depth=3, random_state=0, n_estimators=10)
clf.fit(X, y)

checkpoint_path = './checkpoint.tl'
# Export cuML RF model as Treelite checkpoint
clf.convert_to_treelite_model().to_treelite_checkpoint(checkpoint_path)

/opt/conda/envs/rapids/lib/python3.7/site-packages/cuml/internals/api_decorators.py:794: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)


2. Copy the generated checkpoint file `checkpoint.tl` to another machine on which you'd like to run predictions.

3. On the target machine, install Treelite by running `pip install treelite` or `conda install -c conda-forge treelite`. The machine does not need to have an NVIDIA GPUs and does not need to have cuML installed.

4. You can now load the model from the checkpoint, by running the following on the target machine:

In [15]:
import treelite

# The checkpoint file has been copied over
checkpoint_path = './checkpoint.tl'
tl_model = treelite.Model.deserialize(checkpoint_path)
out_prob = treelite.gtil.predict(tl_model, X, pred_margin=True)
print(out_prob)

[[1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [0.9 0.1 0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [0.9 0.  0.1]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [1.  0.  0. ]
 [0.  1.  0. ]
 [0.  1.  0. ]
 [0.  0.7 0.3]
 [0.  1.  0. ]
 [0.  1.  0. ]
 [0.  1.  0. ]
 [0.  0.9 0.1]
 [0.  0.7 0.3]
 [0.  1.  0. ]
 [0.  1.  0. ]
 [0.  0.9 0.1]
 [0.  1.  0. ]
 [0.  1.  0. ]
 [0.  1.  0. ]
 [0.  1.  0. ]
 [0.  1.  0. ]
 [0.  1.  